In [86]:
from __future__ import print_function
import pandas as pd
import subprocess
import tempfile
import shlex, os


In [74]:
shlex.split('git --work-tree=/Users/wkerzend/tmp/ checkout upstream/pr/8 {cpath}')

['git',
 '--work-tree=/Users/wkerzend/tmp/',
 'checkout',
 'upstream/pr/8',
 '{cpath}']

In [72]:
cd ../

/Users/wkerzend/python/tardis-refdata


In [52]:
def highlight_missing(val):
    if val == True:
        return 'background-color: green'
    else:
        return 'background-color: red'

In [89]:
class ReferenceComparer(object):

    def __init__(self, ref1_hash=None, ref2_hash=None, compare_path='unit_test_data.h5'):
        assert not ((ref1_hash is None) and (ref2_hash is None)), "One hash can not be None"
        self.ref1_hash = ref1_hash
        self.ref2_hash = ref2_hash
        self.compare_path = compare_path
        #subprocess.Popen('git --work-tree=/Users/wkerzend/tmp/ checkout upstream/pr/8 {cpath}', shell=True)
        self.setup()
    
    def setup(self):
        self.tmp_dir = tempfile.mkdtemp()
        print('Created temporary directory at {0}. Delete after use with .teardown'.format(self.tmp_dir))
        for ref_id, ref_hash in enumerate([self.ref1_hash, self.ref2_hash]):
            if ref_hash is not None:
                self._copy_data_from_hash(ref_hash, 'ref{0}_'.format(ref_id))
            else:
                subprocess.Popen('cp {0} {1}'.format(self.compare_path, 
                                                     os.path.join(self.tmp_dir, 
                                                                  '{0}_{1}'.format(ref_id, self.compare_path))), 
                                                     shell=True)
        
    def _copy_data_from_hash(self, ref_hash, prefix):
        git_cmd = ['git']
        git_cmd.append('--work-tree={0}'.format(self.tmp_dir))
        git_cmd += ['checkout', ref_hash, prefix + self.compare_path]
        subprocess.Popen(git_cmd)
        
        #return fname
    def generate_test_table(ref_data1_fname, ref_data2_fname):
        rd1_hdfs = pd.HDFStore(ref_data1_fname, mode='r')
        rd2_hdfs = pd.HDFStore(ref_data2_fname, mode='r')
        rd1_keys = rd1_hdfs.keys()
        rd2_keys = rd2_hdfs.keys()
        rd1_hdfs.close()
        rd2_hdfs.close()
        rd1_df = pd.DataFrame(index=rd1_keys, columns=['exists'])
        rd2_df = pd.DataFrame(index=rd2_keys, columns=['exists'])
        rd1_df['exists'] = True
        rd2_df['exists'] = True
        joined_df = r1_test.join(r2_test, how='outer', lsuffix='_1', rsuffix='_2')
        joined_df = joined_df.fillna(False)
        return joined_df

In [90]:
comparer = ReferenceComparer(ref2_hash='upstream/pr/8')

Created temporary directory at /var/folders/cn/p885gl514_v90bwv3y7q5twr0000gp/T/tmphDxB0i. Delete after use with .teardown


In [82]:
comparer._copy_data_from_hash('upstream/pr/8')

In [54]:
joined_df = generate_test_table('../plasma_reference/reference_data.h5', '../plasma_reference/reference_data.h5')

In [56]:
joined_df.style.applymap(highlight_missing)

,exists_1,exists_2
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/abundance,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/atomic_mass,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/beta_electron,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/beta_rad,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/beta_sobolev,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/delta,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/density,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/electron_densities,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/excitation_energy,False,True
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/f_lu,False,True


In [27]:
r1_test = r1.iloc[:-100]
r2_test = r2.iloc[100:]

In [35]:
r1_test.iloc[0]


exists    True
Name: /_radiative_rates_type_dilute-blackbody/plasma/abundance, dtype: bool

In [40]:
x = r1_test.join(r2_test, how='outer', lsuffix='_1', rsuffix='_2')

In [50]:
x.style.apply(highlight_missing)

ZeroDivisionError: ('integer division or modulo by zero', u'occurred at index exists_1')

In [51]:
%debug

> <ipython-input-49-5bc98d4387a7>(2)highlight_missing()
      1 def highlight_missing(x):
----> 2     1/0

ipdb> x
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/abundance                         False
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/atomic_mass                         NaN
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/beta_electron                       NaN
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/beta_rad                            NaN
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/beta_sobolev                        NaN
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/delta                               NaN
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/density                             NaN
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/electron_densities                  NaN
/_delta_treatment_0.5_helium_treatment_recomb-nlte/plasma/excitation_energy                   NaN
/_d